In [1]:
%matplotlib inline

In [98]:
import xlrd
import os 
import sqlite3
import csv
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd

sns.set(style="whitegrid")

In [99]:
db_path = '../datasets/datawarehouse.db'

# Clean/Query Dataset

In [109]:
con = sqlite3.connect(db_path)
cur = con.cursor()

query = '''
    SELECT NBHDNAME, AVG(ASSESSMENT),
      CASE
        WHEN AVG(ASSESSMENT) >= 1000000 THEN 'high' 
        WHEN AVG(ASSESSMENT) >= 600000 THEN 'medium' 
        ELSE 'low' 
      END AS assessment_tier
    FROM property_info
    WHERE PROPTYPE LIKE '%Residential%'
    GROUP BY NBHDNAME
    ORDER BY AVG(ASSESSMENT) DESC;'''

cur.execute(query)

avg_assess = cur.fetchall()
con.close()

In [326]:
nbhd_dict

{'neighborhood': ['Georgetown, Burleith/Hillandale',
  'Cleveland Park, Woodley Park, Massachusetts Avenue Heights, Woodland-Normanstone Terrace',
  'Woodland/Fort Stanton, Garfield Heights, Knox Hill',
  'Hawthorne, Barnaby Woods, Chevy Chase',
  'Near Southeast, Navy Yard',
  'North Cleveland Park, Forest Hills, Van Ness',
  'Kalorama Heights, Adams Morgan, Lanier Heights',
  'Spring Valley, Palisades, Wesley Heights, Foxhall Crescent, Foxhall Village, Georgetown Reservoir',
  'Capitol Hill, Lincoln Park',
  'Friendship Heights, American University Park, Tenleytown',
  'Colonial Village, Shepherd Park, North Portal Estates',
  'Southwest Employment Area, Southwest/Waterfront, Fort McNair, Buzzard Point',
  'Mayfair, Hillbrook, Mahaning Heights',
  'Edgewood, Bloomingdale, Truxton Circle, Eckington',
  'Capitol View, Marshall Heights, Benning Heights',
  'West End, Foggy Bottom, GWU',
  'Howard University, Le Droit Park, Cardozo/Shaw',
  'Dupont Circle, Connecticut Avenue/K Street',
 

In [110]:
assess_df = pd.DataFrame(avg_assess, columns=['neighborhood', 'price', 'label'])
nbhd_group = []
avg_assessment = []
label = []
for p in avg_assess:
    nbhd_dict[p[0]] = p[2]
    
nbhd_dict['Shaw, Logan Circle'] = 'medium'
nbhd_dict['Downtown, Chinatown, Penn Quarters, Mount Vernon Square, North Capitol Street'] = 'medium'
nbhd_dict['River Terrace, Benning, Greenway, Dupont Park'] = 'medium'
nbhd_dict['Eastland Gardens, Kenilworth'] = 'low'
nbhd_dict['Douglas, Shipley Terrace'] = 'low'

In [316]:
con = sqlite3.connect(db_path)
cur = con.cursor()

query = '''
    SELECT neighbourhood_cleansed, CAST(REPLACE(price, '$', '') AS FLOAT), 
    bathrooms_text,
    CAST(bedrooms AS INT),
    CAST(beds AS INT),
    CAST(accommodates AS INT),
    host_is_superhost,
    host_response_time,
    room_type,
    property_type,
    instant_bookable,
       CAST(REPLACE(host_response_rate, '%', '') AS FLOAT),
       CAST(REPLACE(host_acceptance_rate, '%', '') AS FLOAT),
        CAST(REPLACE(number_of_reviews, '%', '') AS INT),
        CAST(review_scores_rating AS FLOAT),
        CAST(reviews_per_month AS FLOAT)
    FROM listings;'''

cur.execute(query)

listings = cur.fetchall()
con.close()

In [317]:
listings_df = pd.DataFrame(listings, columns = ['neighborhood', 'price', 'bathrooms', 
                                                'bedrooms', 'beds', 'accommodates', 
                                                'host_is_superhost', 'host_response_time',
                                               'room_type', 'property_type', 'instant_bookable',
                                               'host_response_rate', 'host_acceptance_rate', 
                                               'number_of_reviews', 'review_scores_rating',
                                               'reviews_per_month'])
# Create a new column "neighborhood_label" based on the values in the "neighborhood" column
listings_df['neighborhood_label'] = listings_df['neighborhood'].map(nbhd_dict)

In [318]:
# create mapping for shared/private and number of baths

# create new column for shared/private
listings_df['shared_private'] = listings_df['bathrooms'].apply(lambda x: 'private' if 'private' in x.lower() else 'shared')

# create new column for number of baths
listings_df['num_baths'] = listings_df['bathrooms'].apply(lambda x: bath_mapping[listings_df['shared_private'][0]].get(x, None))

listings_df['shared_private'].fillna(listings_df['shared_private'].mode()[0], inplace=True)
listings_df['num_baths'].fillna(listings_df['num_baths'].median(), inplace=True)
listings_df = listings_df.drop('neighborhood', axis=1)

In [319]:
prop_type = {
 'Private room in townhouse': 'house',
'Entire townhouse': 'house',
'Entire rental unit': 'apt',
'Entire guest suite': 'house',
'Room in boutique hotel': 'hotel',
'Entire home': 'house',
'Entire condo': 'apt',
'Private room in bed and breakfast': 'house',
'Private room in home': 'house',
'Private room in rental unit': 'apt',
'Private room in condo': 'apt',
'Entire serviced apartment': 'apt',
'Shared room in hostel': 'hotel',
'Shared room in townhouse': 'house',
'Private room in resort': 'hotel',
'Room in hotel': 'hotel',
'Shared room in rental unit': 'apt',
'Entire guesthouse': 'house',
'Room in bed and breakfast': 'hotel',
'Private room in guest suite': 'house',
'Entire loft': 'apt',
'Entire vacation home': 'house',
'Private room in hostel': 'hotel',
'Room in hostel': 'hotel',
'Shared room in bed and breakfast': 'hotel',
'Shared room in home': 'house',
'Tower': 'unusual',
'Castle': 'unusual',
'Room in aparthotel': 'hotel',
'Private room in guesthouse': 'house',
'Entire place': 'house',
'Private room in loft': 'apt',
'Entire bungalow': 'unusual',
'Private room': 'house',
'Casa particular': 'unusual',
'Private room in villa': 'house',
'Floor': 'apt', 
'Room in serviced apartment': 'apt',
'Tiny home': 'unusual',
'Shared room in guesthouse': 'house',
'Entire cottage': 'unusual',
'Shared room in hotel': 'hotel',
'Camper/RV': 'unusual',
'Houseboat': 'unusual',
'Shared room in loft': 'apt',
'Private room in casa particular': 'unusual',
'Private room in bungalow': 'unusual',
'Private room in serviced apartment': 'apt',
'Tent': 'unusual',
'Campsite': 'unusual',
'Shared room in serviced apartment': 'apt',
'Entire villa': 'unusual',
'Boat': 'unusual'
}

listings_df['property_type'] = listings_df['property_type'].map(prop_type)

In [320]:
# Fill NA variables for superhost, bedrooms, beds, and accommodates
listings_df['bedrooms'].fillna(np.median(listings_df.bedrooms), inplace=True)

listings_df['beds'].fillna(np.median(listings_df.beds), inplace=True)

listings_df['host_is_superhost'].replace('', listings_df.host_is_superhost.mode()[0], inplace=True)

listings_df['host_response_time'].fillna(listings_df.host_response_time.mode()[0], inplace=True)
listings_df['host_response_time'].replace('N/A', listings_df.host_response_time.mode()[0], inplace=True)
listings_df['host_response_time'].replace('', listings_df.host_response_time.mode()[0], inplace=True)

listings_df['room_type'].replace('Hotel room', 'Private room', inplace=True)

listings_df['host_response_rate'].replace('', np.median(listings_df.host_response_rate), inplace=True)

listings_df['review_scores_rating'].replace('', np.mean(listings_df.review_scores_rating), inplace=True)

In [323]:
listings_df.reviews_per_month.max()

59.65

In [324]:
listings_df.to_csv('../datasets/airbnb_selected_variables.csv')

In [325]:
listings_df

,price,bathrooms,bedrooms,beds,accommodates,host_is_superhost,host_response_time,room_type,property_type,instant_bookable,host_response_rate,host_acceptance_rate,number_of_reviews,review_scores_rating,reviews_per_month,neighborhood_label,shared_private,num_baths
0,20.0,1 private bath,1,1,2,f,within an hour,Private room,house,f,0.0,0.0,20,4.85,0.32,medium,private,1.0
1,185.0,2.5 baths,3,3,8,f,within an hour,Entire home/apt,house,f,100.0,100.0,51,4.73,1.87,low,shared,2.5
2,221.0,2 baths,2,2,4,f,within an hour,Entire home/apt,apt,f,99.0,100.0,0,0.00,0.00,medium,shared,2.0
3,142.0,1 bath,1,2,3,t,within an hour,Entire home/apt,house,t,100.0,100.0,11,4.82,1.20,medium,shared,1.0
4,398.0,2 baths,2,3,5,f,within an hour,Entire home/apt,apt,t,0.0,0.0,1,5.00,0.48,high,shared,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8623,70.0,1 bath,0,1,2,f,within a day,Entire home/apt,apt,f,53.0,20.0,0,0.00,0.00,medium,shared,1.0
8624,253.0,1 bath,2,2,5,f,within an hour,Entire home/apt,apt,t,100.0,87.0,0,0.00,0.00,high,shared,1.0
8625,95.0,1 bath,1,1,4,t,within an hour,Entire home/apt,apt,t,100.0,100.0,0,0.00,0.00,high,shared,1.0
8626,180.0,1 bath,1,1,3,f,within an hour,Entire home/apt,apt,t,100.0,87.0,0,0.00,0.00,high,shared,1.0


In [328]:
listings_df.property_type.unique()

array(['house', 'apt', 'hotel', 'unusual'], dtype=object)